In [33]:
import argparse
import git
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from utils import *
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Subtract
import datetime
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

n_classes = 196
n_elements = 300



gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    print(e)


ROOT_DIR = get_git_root(os.getcwd())


car_names = os.listdir(os.path.join(ROOT_DIR, 'dataset'))
labels = random.sample(car_names, k = n_classes)

(trainX, trainY), (testX, testY), (trainX_bbox, testX_bbox) = load_dataset(ROOT_DIR, labels, n_elements, img_resolution=(224,224),
                                                                                 crop=True, greyscale=False, random_sample = False)
trainX = np.asarray(trainX)
trainY = np.asarray(trainY)
testX = np.asarray(testX)
testY = np.asarray(testY)

trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
norm_trainY = normalize_labels(trainY)
norm_testY = normalize_labels(testY)

input_shape = trainX.shape[1:4]


(trainX, trainY) = make_pairs(trainX, norm_trainY)
(testX, testY) = make_pairs(testX, norm_testY)
trainX = [trainX[:, 0], trainX[:, 1]]
testX = [testX[:, 0], testX[:, 1]]



model = tf.keras.applications.VGG16(include_top=False, weights='imagenet',
                            input_shape=input_shape, pooling='max')
for each_layer in model.layers:
    each_layer.trainable = False


inputA = Input(shape=input_shape)
inputB = Input(shape=input_shape)
featsA = model(inputA)
featsB = model(inputB)
# distance = Lambda(utils.euclidean_distance)([featsA, featsB])
# feats = Concatenate()([featsA, featsB])
feats = Subtract()([featsA, featsB])
# feats = Dense(4096, activation='relu', name='Dense_1')(feats)
# output = Dense(4096, activation='relu', name='Dense_2')(feats)
# output = Dense(1, activation="sigmoid")(output)
model = tf.keras.models.Model([inputA, inputB], feats)

In [34]:
trainX_emb = model.predict(trainX)
testX_emb = model.predict(testX)
trainY = np.squeeze(trainY, 1)
testY = np.squeeze(testY, 1)

In [35]:
model.compile()
svm_model = SVC(kernel='rbf', verbose=True)

In [36]:
svm_model.fit(trainX_emb, trainY)

[LibSVM]

SVC(verbose=True)

In [37]:
trainY_preds = svm_model.predict(trainX_emb)
testY_preds = svm_model.predict(testX_emb)
print(accuracy_score(trainY_preds, trainY))
print(accuracy_score(testY_preds, testY))

0.9116169932488062
0.7354543203763307


In [38]:
import pickle
with open('svm_rbf.pickle', 'wb') as file:
    pickle.dump(svm_model, file)

In [39]:
!pwd

/home/alvaro/Vehicle-Model-Recognition/dataset
